In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from seaborn.external.docscrape import header

In [9]:
from dotenv import load_dotenv

load_dotenv()


True

In [10]:
import pandas as pd

In [11]:
books = pd.read_csv("books_cleaned.csv")

In [12]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,description_length,title_and_subtitle,tagged description
0,9.780002e+12,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,199.0,Gilead,9780002005883.0: A NOVEL THAT READERS and crit...
1,9.780002e+12,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,205.0,Spider's Web: A Novel,9780002261982.0: A new 'Christie for Christmas...
2,9.780006e+12,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,57.0,Rage of angels,"9780006178736.0: A memorable, mesmerizing hero..."
3,9.780006e+12,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,45.0,The Four Loves,9780006280897.0: Lewis' work on the nature of ...
4,9.780006e+12,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,75.0,The Problem of Pain,"9780006280934.0: ""In The Problem of Pain, C.S...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9.788172e+12,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,288.0,Mistaken Identity,9788172235222.0: On A Train Journey Home To No...
5193,9.788173e+12,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,63.0,Journey to the East,9788173031014.0: This book tells the tale of a...
5194,9.788180e+12,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,117.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623.0: Wisdom to Create a Life of Pa...
5195,9.788185e+12,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,174.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535.0: This collection of the timele...


In [13]:
books["tagged description"].to_csv("tagged_descriptions.txt", sep="\n", index=False, header=False)


In [19]:
#loading the data into langchain
raw_documents= TextLoader("tagged_descriptions.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1171, which is longer than the specified 0
Created a chunk of size 1217, which is longer than the specified 0
Created a chunk of size 376, which is longer than the specified 0
Created a chunk of size 312, which is longer than the specified 0
Created a chunk of size 486, which is longer than the specified 0
Created a chunk of size 485, which is longer than the specified 0
Created a chunk of size 963, which is longer than the specified 0
Created a chunk of size 191, which is longer than the specified 0
Created a chunk of size 846, which is longer than the specified 0
Created a chunk of size 299, which is longer than the specified 0
Created a chunk of size 200, which is longer than the specified 0
Created a chunk of size 884, which is longer than the specified 0
Created a chunk of size 1091, which is longer than the specified 0
Created a chunk of size 1192, which is longer than the specified 0
Created a chunk of size 307, which is longer than the specified 0
Create

In [20]:
#checking if it has correctly loaded
documents[0]

Document(metadata={'source': 'tagged_descriptions.txt'}, page_content='9780002005883.0: A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details

In [22]:
#create vector database
db_books = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings()
)

In [23]:

query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k= 10)
docs

#recommendation of description

[Document(id='ffb728bd-88c0-4347-8c42-94ccb9c53be8', metadata={'source': 'tagged_descriptions.txt'}, page_content='9780786808069.0: Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='ca811f15-a9e3-4c3a-9644-06bae71463a7', metadata={'source': 'tagged_descriptions.txt'}, page_content="9780786808380.0: Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(id='9eac7350-f9b4-40bd-bb8f-3dec7c2fcfd3', metadata={'so

In [46]:
import re

# Extract numeric ISBN using regex
match = re.search(r'\d{13}', docs[0].page_content.split()[0].strip())
if match:
    isbn = match.group(0)  # this is a 13-digit ISBN string
    isbn = int(isbn)       # convert to int if books["isbn13"] is int
    books_detail = books[books["isbn13"] == isbn]
    print(books_detail)
else:
    print("No valid ISBN found in page content.")

            isbn13      isbn10                                title  \
3747  9.780787e+12  0786808063  Baby Einstein: Neighborhood Animals   

                                authors        categories  \
3747  Marilyn Singer;Julie Aigner-Clark  Juvenile Fiction   

                                              thumbnail  \
3747  http://books.google.com/books/content?id=X9a4P...   

                                            description  published_year  \
3747  Children will discover the exciting world of t...          2001.0   

      average_rating  num_pages  ratings_count  description_length  \
3747            3.89       16.0          180.0                40.0   

                       title_and_subtitle  \
3747  Baby Einstein: Neighborhood Animals   

                                     tagged description  
3747  9780786808069.0: Children will discover the ex...  


In [48]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
        ) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k= 50)

    books_list =[]

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return  books[books["isbn13"].isin(books_list)]

In [50]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
        ) -> pd.DataFrame:

    recs = db_books.similarity_search(query, k=top_k)
    books_list = []

    for rec in recs:
        raw = rec.page_content.strip('"').split()[0]
        clean = raw.replace(":", "").split(".")[0]  # Remove colon and decimal part
        try:
            isbn = int(clean)
            books_list.append(isbn)
        except ValueError:
            print(f"Skipping invalid ISBN: {raw}")

    return books[books["isbn13"].isin(books_list)]

In [51]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,description_length,title_and_subtitle,tagged description
442,9.780068e+12,006757520X,The Sense of Wonder,Rachel Carson,Nature,http://books.google.com/books/content?id=Zee5S...,"First published more than three decades ago, t...",1998.0,4.39,112.0,1160.0,322.0,The Sense of Wonder,9780067575208.0: First published more than thr...
3214,9.780690e+12,0689861133,"Moo, Baa, la la La!",Sandra Boynton,Animal sounds,http://books.google.com/books/content?id=Gz40A...,Children will love joining in and imitating th...,2004.0,4.20,14.0,28261.0,27.0,"Moo, Baa, la la La!",9780689861130.0: Children will love joining in...
3581,9.780764e+12,0763620874,Judy Moody Saves the World!,Megan McDonald,Juvenile Fiction,http://books.google.com/books/content?id=xDIRB...,When Judy Moody gets serious about protecting ...,2004.0,4.03,160.0,5883.0,34.0,Judy Moody Saves the World!,9780763620875.0: When Judy Moody gets serious ...
3747,9.780787e+12,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,40.0,Baby Einstein: Neighborhood Animals,9780786808069.0: Children will discover the ex...
3748,9.780787e+12,0786808373,Baby Einstein: Birds,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=0jxHP...,"Introducing your baby to birds, cats, dogs, an...",2002.0,3.78,20.0,9.0,64.0,Baby Einstein: Birds,9780786808373.0: Introducing your baby to bird...
3749,9.780787e+12,0786808381,Baby Einstein: Babies,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=jv4NA...,"Introduce your babies to birds, cats, dogs, an...",2002.0,4.03,20.0,29.0,65.0,Baby Einstein: Babies,9780786808380.0: Introduce your babies to bird...
3750,9.780787e+12,078680839X,Baby Einstein: Dogs,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=qut8t...,"Introduce your baby to birds, cats, dogs, and ...",2002.0,3.81,20.0,26.0,64.0,Baby Einstein: Dogs,"9780786808397.0: Introduce your baby to birds,..."
3751,9.780787e+12,0786808713,Baby Einstein: What Does Violet See? Raindrops...,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=95IIA...,A very special puddle sets Violet the mouse of...,2002.0,3.25,18.0,16.0,52.0,Baby Einstein: What Does Violet See? Raindrops...,9780786808717.0: A very special puddle sets Vi...
3760,9.780787e+12,0786812915,The Big Box,Toni Morrison;Slade Morrison,Juvenile Fiction,http://books.google.com/books/content?id=LyYKA...,"In her first illustrated book for children, th...",2002.0,3.95,48.0,375.0,28.0,The Big Box,9780786812912.0: In her first illustrated book...
3797,9.780789e+12,0789458209,Tree,David Burnie,Juvenile Nonfiction,http://books.google.com/books/content?id=Qwsqj...,Photographs and text explore the anatomy and l...,2000.0,4.07,64.0,5.0,37.0,Tree,9780789458209.0: Photographs and text explore ...
